In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from scipy.stats import scoreatpercentile

# Descargar el archivo comprimido desde GitHub
!wget https://github.com/watorres/ProyectoDeepLearning/raw/main/device_gnss.csv.zip

# Descomprimir el archivo
import zipfile
with zipfile.ZipFile('device_gnss.csv.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('device_gnss.csv')

# Seleccionar las columnas de entrada y salida
X = df[['Cn0DbHz', 'PseudorangeRateMetersPerSecond', 'PseudorangeRateUncertaintyMetersPerSecond']]
y = df[['SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos de entrada
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape de los datos para la entrada en la red convolucional
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Definir la arquitectura de la red convolucional
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))  # Salida con 3 dimensiones (x, y, z)

# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer='adam')

# Entrenar el modelo
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(X_test_reshaped)

# Calcular los errores de distancia horizontal
dist_errors = np.linalg.norm(y_pred - y_test, axis=1)

# Calcular los errores de percentil 50 y 95
percentile_50 = np.percentile(dist_errors, 50)
percentile_95 = np.percentile(dist_errors, 95)

# Imprimir los resultados
print('Error de distancia horizontal:', dist_errors)
print('Error de percentil 50:', percentile_50)
print('Error de percentil 95:', percentile_95)

--2023-05-31 22:27:52--  https://github.com/watorres/ProyectoDeepLearning/raw/main/device_gnss.csv.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/watorres/ProyectoDeepLearning/main/device_gnss.csv.zip [following]
--2023-05-31 22:27:53--  https://raw.githubusercontent.com/watorres/ProyectoDeepLearning/main/device_gnss.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9790985 (9.3M) [application/zip]
Saving to: ‘device_gnss.csv.zip’

device_gnss.csv.zip 100%[===================>]   9.34M  --.-KB/s    in 0.1s    

2023-05-31 22:27:53 (76.4 MB/s) - ‘device_gnss.csv.zip’ saved [9790985/979

**El resultado que obtenido muestra que el modelo no ha convergido durante el entrenamiento. Se observa que tanto la pérdida de entrenamiento (loss) como la pérdida de validación (val_loss) tienen valores nan (no numéricos), lo que indica que no se ha logrado calcular una función de pérdida válida durante el entrenamiento.**

**A continuación, se modifica el código en el cual se han realizado los siguientes cambios:**

**1) Se ha ajustado la tasa de aprendizaje (learning_rate) del optimizador Adam a 0.001. Puedes ajustar este valor según sea necesario.**

**2) El número de épocas de entrenamiento se ha incrementado a 50. Esto permite que el modelo tenga más oportunidades de aprender durante el entrenamiento.**

**3) Se ha agregado un optimizer personalizado al compilar el modelo, utilizando el valor ajustado para la tasa de aprendizaje.**

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from scipy.stats import scoreatpercentile

# Descargar el archivo comprimido desde GitHub
!wget https://github.com/watorres/ProyectoDeepLearning/raw/main/device_gnss.csv.zip

# Descomprimir el archivo
import zipfile
with zipfile.ZipFile('device_gnss.csv.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('device_gnss.csv')

# Seleccionar las columnas de entrada y salida
X = df[['Cn0DbHz', 'PseudorangeRateMetersPerSecond', 'PseudorangeRateUncertaintyMetersPerSecond']]
y = df[['SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos de entrada
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape de los datos para la entrada en la red convolucional
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Definir la arquitectura de la red convolucional
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))  # Salida con 3 dimensiones (x, y, z)

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Entrenar el modelo
model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(X_test_reshaped)

# Calcular los errores de distancia horizontal
dist_errors = np.linalg.norm(y_pred - y_test, axis=1)

# Calcular los errores de percentil 50 y 95
percentile_50 = np.percentile(dist_errors, 50)
percentile_95 = np.percentile(dist_errors, 95)

# Imprimir los resultados
print('Error de distancia horizontal:', dist_errors)
print('Error de percentil 50:', percentile_50)
print('Error de percentil 95:', percentile_95)

--2023-05-31 22:34:11--  https://github.com/watorres/ProyectoDeepLearning/raw/main/device_gnss.csv.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/watorres/ProyectoDeepLearning/main/device_gnss.csv.zip [following]
--2023-05-31 22:34:11--  https://raw.githubusercontent.com/watorres/ProyectoDeepLearning/main/device_gnss.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9790985 (9.3M) [application/zip]
Saving to: ‘device_gnss.csv.zip.1’

device_gnss.csv.zip 100%[===================>]   9.34M  --.-KB/s    in 0.1s    

2023-05-31 22:34:11 (74.7 MB/s) - ‘device_gnss.csv.zip.1’ saved [9790985

**De  nuevo, el resultado que obtenido muestra que el modelo no ha convergido durante el entrenamiento. Se observa que tanto la pérdida de entrenamiento (loss) como la pérdida de validación (val_loss) tienen valores nan (no numéricos), lo que indica que no se ha logrado calcular una función de pérdida válida durante el entrenamiento.**